In [55]:
!python -m spacy download en_core_web_lg

     ---------------------------------------- 0.0/400.7 MB ? eta -:--:--
     ---------------------------------------- 1.8/400.7 MB 9.1 MB/s eta 0:00:44
     --------------------------------------- 4.2/400.7 MB 10.5 MB/s eta 0:00:38
      --------------------------------------- 6.0/400.7 MB 9.7 MB/s eta 0:00:41
      -------------------------------------- 8.4/400.7 MB 10.0 MB/s eta 0:00:40
     - ------------------------------------ 10.7/400.7 MB 10.3 MB/s eta 0:00:38
     - ------------------------------------ 13.6/400.7 MB 10.8 MB/s eta 0:00:36
     - ------------------------------------ 16.0/400.7 MB 10.9 MB/s eta 0:00:36
     - ------------------------------------ 18.4/400.7 MB 10.9 MB/s eta 0:00:36
     - ------------------------------------ 21.0/400.7 MB 11.0 MB/s eta 0:00:35
     -- ----------------------------------- 23.3/400.7 MB 11.1 MB/s eta 0:00:35
     -- ----------------------------------- 26.0/400.7 MB 11.2 MB/s eta 0:00:34
     -- ----------------------------------- 28.

In [56]:
import spacy

nlp = spacy.load("en_core_web_lg")
doc = nlp("India beat Bangladesh")
doc.ents
print(nlp.pipe_names)

from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


In [57]:
import json

with open('dataset/sample.json', 'r') as f:
    data = json.load(f)
print(data)

[{'content': "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\nâ€¢ To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendri

In [58]:
training_data = []
for example in data:
    temp_dict = {}
    temp_dict['text'] = example['content']
    temp_dict['entities'] = []
    for annotation in example['annotation']:
        for point in annotation['points']:  
            start = point['start']
            end = point['end']
            tag = annotation['label']
            #print("".join(tag).upper())
            label="".join(tag).upper()
            temp_dict['entities'].append((start, end, label))
    training_data.append(temp_dict)

print(training_data)

[{'text': "Abhishek Jha\nApplication Development Associate - Accenture\n\nBengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a\n\nâ€¢ To work for an organization which provides me the opportunity to improve my skills\nand knowledge for my individual and company's growth in best possible ways.\n\nWilling to relocate to: Bangalore, Karnataka\n\nWORK EXPERIENCE\n\nApplication Development Associate\n\nAccenture -\n\nNovember 2017 to Present\n\nRole: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries\nfor the Bot which will be triggered based on given input. Also, Training the bot for different possible\nutterances (Both positive and negative), which will be given as\ninput by the user.\n\nEDUCATION\n\nB.E in Information science and engineering\n\nB.v.b college of engineering and technology -  Hubli, Karnataka\n\nAugust 2013 to June 2017\n\n12th in Mathematics\n\nWoodbine modern school\n\nApril 2011 to March 2013\n\n10th\n\nKendriya 

In [59]:
 from spacy.tokens import DocBin
 from tqdm import tqdm

 nlp = spacy.blank("en")
 doc_bin = DocBin()

 from spacy.util import filter_spans

 for training_example in tqdm(training_data):
     text = training_example['text']
     labels = training_example['entities']
     doc = nlp.make_doc(text)
     ents = []
     for start, end, label in labels:
         span = doc.char_span(start, end, label=label, alignment_mode="contract")
         if span is None:
             #print("Skipping entity")
             pass
         else:
             ents.append(span)
     filtered_ents = filter_spans(ents)
     doc.ents = filtered_ents
     doc_bin.add(doc)

 doc_bin.to_disk("train.spacy")


100%|██████████| 1/1 [00:00<00:00, 166.65it/s]


In [60]:
!python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [61]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy 



ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
⚠ Aborting and saving the final best model. Encountered exception:
ValueError("[E024] Could not find an optimal move to supervise the parser.
Usually, this means that the model can't be updated in a way that's valid and
satisfies the correct annotations specified in the GoldParse. For example, are
all labels added to the model? If you're training a named entity recognizer,
also make sure that none of your annotated entity spans have leading or trailing
whitespace or punctuation. You can also use the `debug data` command to validate
your JSON-formatted training data. For det

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Rohit\Python\python-3.11.4-embed-amd64\Python\Lib\site-packages\spacy\__main__.py", line 4, in <module>
    setup_cli()
  File "c:\Rohit\Python\python-3.11.4-embed-amd64\Python\Lib\site-packages\spacy\cli\_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "c:\Rohit\Python\python-3.11.4-embed-amd64\Python\Lib\site-packages\click\core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Rohit\Python\python-3.11.4-embed-amd64\Python\Lib\site-packages\typer\core.py", line 728, in main
    return _main(
           ^^^^^^
  File "c:\Rohit\Python\python-3.11.4-embed-amd64\Python\Lib\site-packages\typer\core.py", line 197, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "c:\Rohit\Python\python-3.11.4-embed-amd64\Python\Lib\site-packages\click\co

In [62]:
!python -m spacy debug data --help

                                                                               
 Usage: python -m spacy debug data [OPTIONS] CONFIG_PATH                       
                                                                               
 Analyze, debug and validate your training and development data. Outputs       
 useful stats, and can help you find problems like invalid entity annotations, 
 cyclic dependencies, low data labels and more.                                
 DOCS: https://spacy.io/api/cli#debug-data                                     
                                                                               
┌─ Arguments ─────────────────────────────────────────────────────────────────┐
│ *    config_path      PATH  Path to config file [default: None] [required]  │
└─────────────────────────────────────────────────────────────────────────────┘
┌─ Options ───────────────────────────────────────────────────────────────────┐
│ --code-path,--code  -c       PATH  Pat

In [63]:
nlp_ner = spacy.load("model-best")
doc = nlp_ner("Manisha Bharti\nSoftware Automation Engineer\n\nPune, Maharashtra - Email me on Indeed: indeed.com/r/Manisha-Bharti/3573e36088ddc073\n\n• 3.5 years of professional IT experience in Banking and Finance domain and currently working\nas Software")


spacy.displacy.render(doc, style="ent", jupyter=True)

OSError: [E050] Can't find model 'model-best'. It doesn't seem to be a Python package or a valid path to a data directory.